<a href="https://colab.research.google.com/gist/KardelRuveyda/a9c0e02f3a862fc5b7ef8e6281e43068/huggingface-finetuning-exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch




# Excel dosyasını oku
df = pd.read_excel("train-report.xlsm")  # Excel dosyanızın adını burada belirtin

# Verileri bir metin dosyasına kaydedin
df.to_csv('veri_kumesi.txt', index=False, header=False, sep='\t', encoding='utf-8')


In [ ]:
# "cümle" ve "intentadi" sütunlarını seçin
cumleler = df["Cumle"].astype(str).tolist()
intentler = df["IntentAdi"].astype(str).tolist()


print(cumleler);
print(intentler);

['53293112028', 'ruveydakardelcetin@gmail.com', 'maraba', 'kardel.cetin@d-teknoloji.com.tr', 'aksam', 'aafasfwef', '15.02.2000', 'dsgadsgf', 'aasdada', 'aaa', 'asd', 'ccc', 'bodrum canli', 'anlayamadım', 'ne demek istediğinizi anlamadım', 'ana menüye dön', 'ana menü', 'merhaba', 'mrb', 'naber', 'sa', 'merhabalar', 'selamlar', 'merhaba', 'Selam', 'selamün aleyküm', 'merhaba canım', 'Merhaba', 'Selamın aleyküm', 'selamın aleyküm', 'ne demek istiyorsunuz?', 'bay bay', 'görüşürüz', 'görüşmek üzere', 'by', 'bye', 'görüşrüüz', 'güle güle', 'bayy', 'iyi akşamlar', 'iyi günler', 'eyvallah', 'YNT SİGORTA ACENTELİĞİ LTD. ŞTİ.', 'YAY10', 'YAY6', 'Yatarak', 'satın alma yapmak isityorum', 'satın al', 'satış gerçekleştirmek istiyorum', 'sigorta yaptırmadım', 'sigorta yaptırmak istiyorum', 'teklifi satın almak istiyorum', 'sigortam mevcut değil adres bilgilerini girmek istiyorum', 'sigortam yok', 'sigortalı değilim', 'teklifi satışa dönüştürmek istiyorum', '5305153061', 'evet', 'evettt', 'evt', 'olur

In [ ]:
#redrussianarmy/gpt2-turkish-cased
model_name = "redrussianarmy/gpt2-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# Verileri tokenlara çevirme
encoding = tokenizer(cumleler, return_tensors="pt", padding=True, truncation=True)

In [ ]:
# Veri kümesini oluşturma
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="veri_kumesi.txt",  # Veri kümesi dosyasının yolunu belirtin
    block_size=128,  # Metin bloğunun boyutunu belirleyin
)

# Veri kümesindeki örnek sayısını alın
num_examples = len(dataset)

print(num_examples)

# İlk 10 örneği görüntüle
for i in range(min(10, num_examples)):  # Eğer veri kümesi 10'dan az örnek içeriyorsa, tüm örnekleri görüntüleyin
    print(dataset[i])



16
tensor([ 4310,  1959,  3132, 10232,  2078,   197,    67,  8254,   198,   622,
         3304,    67,   461,   446,   417,    66,   316,   259,    31, 14816,
           13,   785,   197,    67,  8254,   198,  3876, 15498,   197,   741,
        43695,  1689,   198,    74,   446,   417,    13,    66,   316,   259,
           31,    67,    12,   660, 15418, 14057,  7285,    13,   785,    13,
         2213,   197,    67,  8254,   198,  4730,   321,   197,    67,  8254,
          198,    64,  1878,   292,    69,   732,    69,   197,    67,  8254,
          198,  1314,    13,  2999,    13, 11024,   197,    67,  8254,   198,
         9310,    70,  5643,    70,    69,   197,    67,  8254,   198,    64,
          292,    67,  4763,   197,    67,  8254,   198, 46071,   197,    67,
         8254,   198,   292,    67,   197,    67,  8254,   198,   535,    66,
          197,    67,  8254,   198,    65,   375,  6582,   460,  4528,   197,
           67,  8254,   198,   272, 10724,   321,   324, 3010

In [ ]:
# Eğitim için veri koleksiyonunu hazırlama
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Dil modellemesi için maskleme (masked language modeling) kullanmıyoruz
)


In [ ]:
# Eğitim argümanlarını ayarlama
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Eğitim döngüsü sayısı
    per_device_train_batch_size=8,
    save_steps=10_000,  # Her bu kadar adımda bir modeli kaydet
    save_total_limit=2,  # Kaç model saklanacağını belirle
    evaluation_strategy="steps",
    eval_steps=10_000,  # Her bu kadar adımda bir değerlendirme yap
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
    logging_dir="./logs",
)


In [ ]:
# Trainer oluşturma
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# Fine-tuning işlemini başlatma
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=6, training_loss=6.126461664835612, metrics={'train_runtime': 74.419, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.081, 'total_flos': 3135504384000.0, 'train_loss': 6.126461664835612, 'epoch': 3.0})

In [ ]:
prompt = "Sigorta botu musun?"  # Sigorta botuna sormak istediğiniz soruyu burada belirtin

# Prompt'ı tokenize edin
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Modeli kullanarak cevap alın
with torch.no_grad():
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Cevabı decode edin
response = tokenizer.decode(output[0], skip_special_tokens=True)

# Cevabı yazdırın
print("Sigorta Botu Cevabı:", response)




Sigorta Botu Cevabı: Sigorta botu musun?
Bu sitede yer alan bilgiler, yorum ve tavsiyeler, yorum ve tavsiyeler, yorum ve tavsiyede bulunanların kişisel görüşlerine dayanmaktadır. Bu sitede yer alan bilgiler, yorum ve tavsiyede bulunanların kişisel görüşlerine dayanmaktadır. Bu sitede yer alan bilgiler, yorum


In [ ]:
# Sorunuzu veya ifadenizi belirtin
soru = "Sen bir sigorta botu musun?"

# Soruyu tokenize edin
input_ids = tokenizer.encode(soru, return_tensors="pt")

# Modeli kullanarak cevap üretin
output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=50256)

# Cevabı çıkartın ve ekrana yazdırın
cevap = tokenizer.decode(output[0], skip_special_tokens=True)
print(cevap)

Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu musun? Sen bir sigorta botu
